## Connect to the Reddit API

To Connect to Reddit API, we must have :
- account (username & password)
- go to [reddit api documentation](https://www.reddit.com/dev/api/)
    - Follow the instruction to connect to the API [ access rules](https://github.com/reddit-archive/reddit/wiki/API)
    - On Rules [OAuth2](https://github.com/reddit-archive/reddit/wiki/OAuth2) we have:
        - The lik to get the web_app & secret code [https://www.reddit.com/prefs/apps](https://www.reddit.com/prefs/apps)
    - On [Quick Start Example](https://github.com/reddit-archive/reddit/wiki/OAuth2-Quick-Start-Example) we have :
        - The API url : [https://www.reddit.com/api/v1/access_token](https://www.reddit.com/api/v1/access_token)
        - An exemple how to connect with request method [Python example](https://github.com/reddit-archive/reddit/wiki/OAuth2-Quick-Start-Example#python-example)

In [1]:
import json

import requests
import requests.auth
from pathlib import Path

In [2]:
path = Path.cwd().parent / "secret"

def open_file(file_name):
    """
    :param file_name: must be json file
    :return: load json file on variable
    """
    with open(path / file_name, "r") as f:
        return json.load(f)

user_name = open_file("user_name.json")
reddit_key = open_file("reddit_key.json")

In [6]:
user = list(reddit_key.keys())[0]
secret = list(reddit_key.values())[0]
username =list(user_name.keys())[0]
password = list(user_name.values())[0]

In [7]:
client_auth = requests.auth.HTTPBasicAuth(user,secret)
post_data={"grant_type":"password","username":username, "password":password}
headers={"User-agent":"FORMATION"}

In [12]:
response = requests.post('https://www.reddit.com/api/v1/access_token',
                    auth=client_auth, data=post_data, headers=headers)
print(response.status_code)

200


In [18]:
access_token = 'bearer ' + response.json()["access_token"]

In [19]:
headers = {"authorization" : access_token, "User-agent" : "FORMATION"}
params = {"t" : "day"} # to get the last day
resp = requests.get("https://oauth.reddit.com/r/python/top", headers=headers, params=params).json()
print(resp)

{'kind': 'Listing', 'data': {'after': None, 'dist': 19, 'modhash': None, 'geo_filter': '', 'children': [{'kind': 't3', 'data': {'approved_at_utc': None, 'subreddit': 'Python', 'selftext': 'Hey!\n\nFor those who want the short version:\n\nThis is the first PyGame I’ve made by myself! It\'s a solitaire like game. Here’s the video that shows the program: [https://youtu.be/Yb04u2Ggdgc](https://youtu.be/Yb04u2Ggdgc)\n\nAnd here\'s [my code](https://github.com/tlb0/EgoGame) :)\n\nFor those who want to read about my process:\n\nIn the past few weeks, I’ve been learning how to code with python. I\'ve already made some simple projects like tic tac toe, hangman or rock paper scissors but they\'re confined to the terminal. I decided to create a game that uses a real user interface for once. I followed the [spaceship game tutorial by Tech by Tim](https://www.youtube.com/watch?v=jO6qQDNa2UY&amp;t=221s&amp;ab_channel=TechWithTim) and figured that i was ready to start my own project.\n\nThe game i ch